<a href="https://colab.research.google.com/github/KayalvizhiT513/Gradient_Descent_Comparison/blob/main/SGD_Adagrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adagrad (Adaptive Gradient Algorithm)**
Whatever the optimizer we learned till SGD with momentum, the learning rate remains constant. In Adagrad optimizer, there is no momentum concept so, it is much simpler compared to SGD with momentum.

The idea behind Adagrad is to use different learning rates for each parameter base on iteration. The reason behind the need for different learning rates is that the learning rate for sparse features parameters needs to be higher compare to the dense features parameter because the frequency of occurrence of sparse features is lower.

* Equation:

![link text](https://miro.medium.com/v2/resize:fit:828/format:webp/1*XWvo73EMLhIeGs35xkimVw.png)

In the above Adagrad optimizer equation, the learning rate has been modified in such a way that it will automatically decrease because the summation of the previous gradient square will always keep on increasing after every time step.





In [2]:
import numpy as np
import csv

# Step 1: Initialize weights and learning rate
w_0 = 0.8260560647266798
w_1 = 0.5782539087214469
learning_rate = 0.01

# Step 2: Load data from CSV file
data = np.genfromtxt('randXY.csv', delimiter=',', skip_header=1)  # Adjust the filename accordingly

# Extract X and Y from the loaded data
X = data[:, 0]  # Assuming the first column is X
Y = data[:, 1]  # Assuming the second column is Y

print("Initial w0: ", w_0, "\n Initial w1: ", w_1)

Initial w0:  0.8260560647266798 
 Initial w1:  0.5782539087214469


In [3]:
def percentage_difference(value1, value2):
    return (np.abs(value1 - value2) / ((value1 + value2) / 2)) * 100

In [11]:
import matplotlib.pyplot as plt

# Function to plot epoch vs. loss
def plot_loss_vs_epoch(loss_history, algorithm_name):
    plt.plot(range(len(loss_history)), loss_history, label=algorithm_name)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'{algorithm_name} - Epoch vs. Loss')
    plt.legend()
    plt.show()

In [34]:
from math import sqrt

def update_w_with_adagrad(w, learning_rate, a, gradient_w, e = 1e-8):
    # Update learning_rate
    learning_rate = learning_rate / sqrt(a + e)
    print(learning_rate)
    # Update w0
    w = w - (learning_rate * gradient_w)

    return w, learning_rate

In [38]:
# Step 3: Define SGD function
def sgd_one_sample(X, Y, w0, w1, learning_rate, epochs=6000, tol=1, consecutive_instances=10):
    n = len(X)
    prev_loss = float('inf')
    count = 0
    a = 0

    # Initialize previous gradients and accumulators
    learning_rate_w0 = 0.01
    learning_rate_w1 = 0.01
    a_w0 = 0.0
    a_w1 = 0.0

    for epoch in range(epochs):
        for i in range(n):
            # Select one random data point
            random_index = np.random.randint(0, n)
            x_i = X[random_index]
            y_i = Y[random_index]

            # Calculate prediction and loss for the selected point
            prediction = w0 + w1 * x_i
            loss = (y_i - prediction)**2

            # Calculate gradients
            gradient_w0 = -2 * (y_i - prediction)
            gradient_w1 = -2 * (y_i - prediction) * x_i

            # Update weights
            w0, learning_rate_w0 = update_w_with_adagrad(w0, learning_rate_w0, a_w0, gradient_w0)
            w1, learning_rate_w1 = update_w_with_adagrad(w1, learning_rate_w1, a_w1, gradient_w1)

            # Update a for next iteration
            a_w0 += gradient_w0**2
            a_w1 += gradient_w1**2

            if learning_rate_w0 == 0 and learning_rate_w1 == 0:
              break


        # Calculate overall loss for monitoring
        predictions = w0 + w1 * X
        overall_loss = np.mean((Y - predictions)**2)

        percent_diff = percentage_difference(prev_loss, overall_loss)
        if percent_diff < tol:
          count += 1
        else:
          count = 0

        # Print loss for monitoring
        if epoch % 500 == 0:
            #print(f"learning_rate_w0: {learning_rate_w0}, learning_rate_w1: {learning_rate_w1}")
            print(f"Epoch {epoch}, Loss: {overall_loss}")

        # the model will stop learning as the learning rate is 0
        if learning_rate_w0 == 0 or learning_rate_w1 == 0:
            print(f"learning_rate_w0: {learning_rate_w0}, learning_rate_w1: {learning_rate_w1}")
            print(f"Epoch {epoch}, Loss: {overall_loss}")
            break

        # Update the stopping criteria to consider non-inf values
        if count >= consecutive_instances:
            print(f"Epoch {epoch}, Loss: {overall_loss}")
            print("Converged! ", count)
            break

        # Append loss to the history
        loss_history_sgd_one_sample.append(overall_loss)

        # Update previous loss for the next iteration
        prev_loss = overall_loss

    return w0, w1

# Step 4: Run SGD with one training sample at a time
w_0 = 0.8260560647266798
w_1 = 0.5782539087214469
learning_rate = 0.01
loss_history_sgd_one_sample = []
w0_sgd_one_sample, w1_sgd_one_sample = sgd_one_sample(X, Y, w_0, w_1, learning_rate)
print(f"Final weights for SGD with one sample: w0={w0_sgd_one_sample}, w1={w1_sgd_one_sample}")

100.0
100.0
8.018772554696074
15.536575201001547
0.0022840446933006784
0.005523322290864387
1.892654009386141e-08
5.899913182051107e-08
1.1181819680776095e-13
4.513205753698324e-13
6.094769735562881e-19
3.4311821889024295e-18
3.0274287262834344e-24
2.5480211482889948e-23
1.4356865868997696e-29
1.8896355452198665e-28
5.899617538607427e-35
1.1361879769379344e-33
2.10912605771523e-40
5.269730068598672e-39
7.263851893891709e-46
2.4309036631357446e-44
2.296604388333853e-51
1.00676883063019e-49
7.036539799001914e-57
4.145923715989854e-55
2.1020928047202692e-62
1.7024314554212093e-60
6.140950831065224e-68
6.976712779307391e-66
1.7658581899542143e-73
2.8583046179271997e-71
4.909330379826512e-79
1.1542999680568876e-76
1.305860835741732e-84
4.4759034612554257e-82
3.320344359727879e-90
1.6493155011326462e-87
8.295256860649703e-96
6.0634491015656925e-93
2.0342622010840526e-101
2.221522652812938e-98
4.7420847019841893e-107
7.398823775109126e-104
1.0883871033225615e-112
2.458438780830822e-109
2.3856

<ipython-input-3-d4fdf7de7d20>:2: RuntimeWarning: invalid value encountered in double_scalars
  return (np.abs(value1 - value2) / ((value1 + value2) / 2)) * 100


Lastly, despite not having to manually tune the learning rate there is one huge disadvantage i.e due to monotonically decreasing learning rates, at some point in time step, the model will stop learning as the learning rate is almost close to 0.